
# Data preprocessing

In [4]:
%cd ..

/Users/fulco/Desktop/recommender-systems-challenge


In [9]:
import numpy as np
import scipy.sparse as sps
from sklearn import feature_extraction

from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from reader import load_urm, load_icm, load_target
from evaluate import _get_instance

ModuleNotFoundError: No module named 'Recommenders.Hybrids.HybridSimilarity_ElasticNet_BPR'

In [ ]:
res_dir = 'result_experiments/csv'
output_root_path = "../result_experiments/"


In [ ]:
URM_all, user_id_unique, item_id_unique = load_urm()

ICM_channel = load_icm("data_ICM_channel.csv", weight=1)
ICM_event = load_icm("data_ICM_event.csv", weight=1)
ICM_genre = load_icm("data_ICM_genre.csv", weight=1)
ICM_subgenre = load_icm("data_ICM_subgenre.csv", weight=1)

ICM_all = sps.hstack([ICM_genre, ICM_subgenre, ICM_channel, ICM_event]).tocsr()
ICMs = [ICM_genre, ICM_subgenre, ICM_channel, ICM_event, ICM_all]

In [ ]:

URM_train, URM_test = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.90)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train, train_percentage = 0.80)

# evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])


#print(n_users)

In [ ]:
n_users = 13650
n_item = 18059

In [ ]:
profile_length = np.ediff1d(sps.csr_matrix(URM_train).indptr)
print("profile", profile_length, profile_length.shape)

block_size = int(len(profile_length)*0.5)
print("block_size",block_size)

sorted_users = np.argsort(profile_length)
print("sorted users", sorted_users)

In [ ]:
import matplotlib.pyplot as plt
counts, bins = np.histogram(profile_length)
plt.hist(bins[:-1], bins, weights=counts)
bins
counts

In [ ]:
for group_id in range(0, 2):
    start_pos = group_id * block_size
    end_pos = min((group_id+1) * block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))

In [ ]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender

MAP_recommender_per_group = {}

collaborative_recommender_class = {"TopPop": TopPop,
                                   "UserKNNCF": UserKNNCFRecommender,
                                   "ItemKNNCF": ItemKNNCFRecommender,
                                   "P3alpha": P3alphaRecommender,
                                   "RP3beta": RP3betaRecommender,
                                   "SLIMElasticNetRecommender": SLIMElasticNetRecommender,
                                   "PureSVDRecommender": PureSVDRecommender,
                                   }

#content_recommender_class = {"ItemKNNCBF": ItemKNNCBFRecommender,
#                             "ItemKNNCFCBF": ItemKNN_CFCBF_Hybrid_Recommender
#                            }

recommender_object_dict = {}

for label, recommender_class in collaborative_recommender_class.items():

            recommender_object = recommender_class(URM_train)

            if isinstance(recommender_object, ItemKNNCFRecommender):
                fit_params = {"topK": 200, "shrink": 200, "feature_weighting": "TF-IDF"}
            elif isinstance(recommender_object, IALSRecommender):
                fit_params = {'num_factors': 55, 'epochs': 50, 'confidence_scaling': 'log',
                              'alpha': 0.06164752624981533, 'epsilon': 0.21164021855039056, 'reg': 0.002507116338282967}
            elif isinstance(recommender_object, RP3betaRecommender):
                fit_params = {'topK': 40, 'alpha': 0.4208737801266599, 'beta': 0.5251543657397256,
                              'normalize_similarity': True}
            elif isinstance(recommender_object, PureSVDRecommender):
                fit_params = {'num_factors' :29}    
            else:
                fit_params = {}     

            if isinstance(recommender_object, SLIMElasticNetRecommender):
              recommender_object.load_model(output_root_path, file_name="slimelastic_urmall.zip")
            else:
              recommender_object.fit(**fit_params)

            recommender_object_dict[label] = recommender_object

# for label, recommender_class in content_recommender_class.items():
#    recommender_object = recommender_class(URM_train, ICM_genres)
#    recommender_object.fit()
#    recommender_object_dict[label] = recommender_object


In [ ]:
%ls

In [ ]:
cutoff = 10

for group_id in range(0, 2):
    
    start_pos = group_id*block_size
    end_pos = min((group_id+1)*block_size, len(profile_length))
    
    users_in_group = sorted_users[start_pos:end_pos]
    
    users_in_group_p_len = profile_length[users_in_group]
    
    print("Group {}, #users in group {}, average p.len {:.2f}, median {}, min {}, max {}".format(
        group_id, 
        users_in_group.shape[0],
        users_in_group_p_len.mean(),
        np.median(users_in_group_p_len),
        users_in_group_p_len.min(),
        users_in_group_p_len.max()))
    
    
    users_not_in_group_flag = np.isin(sorted_users, users_in_group, invert=True)
    users_not_in_group = sorted_users[users_not_in_group_flag]
    
    evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    
    for label, recommender in recommender_object_dict.items():
        result_df, _ = evaluator_test.evaluateRecommender(recommender)
        if label in MAP_recommender_per_group:
            MAP_recommender_per_group[label].append(result_df.loc[cutoff]["MAP"])
        else:
            MAP_recommender_per_group[label] = [result_df.loc[cutoff]["MAP"]]

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline  

_ = plt.figure(figsize=(16, 9))
for label, recommender in recommender_object_dict.items():
    results = MAP_recommender_per_group[label]
    plt.scatter(x=np.arange(0,len(results)), y=results, label=label)
plt.ylabel('MAP')
plt.xlabel('User Group')
plt.legend()
plt.show()